# Do it! BERT와 GPT로 배우는 자연어 처리
### 저자 이기창

# 바이트 페어 인코딩(Byte Parir Encoding : BPE)
* 본래 정보를 압축하는 알고리즘으로 제안 -> 최근 NLP 모델에서 널리 쓰이는 토큰화 기법
* 데이터에서 가장 많이 등장한 무자열을 병합해서 데이터를 압축하는 기법

> aaabdaabac<br>

<br>
데이터에 등장한 글자(a,b,c,d)를 초기 사전으로 구성하며, 연속된 두 글자를 한 글자로 병합<br>
aa-> Z로 병합<br>
<br>

> ZabdZabac



<br>
ab가 다음으로 많이 나왔으므로 ab -> Y로 병합<br>
<br>

> ZYdZYac

<br>
ZY가 다음으로 많이 나왔으므로 ZY -> X로 병합(이미 병합된 문자열도 다시 병합)<br>
<br>

> XdXac

<br>

#### 사전
(a,b,c,d) -> (a,b,c,d,Z,Y,X)<br><br>
5개 -> 7개<br>
len(데이터) 11 -> 5<br>
<br>
* BPE 기반 토큰화 기법은corpus에서 자주 등장하는 문자열(Subword)을 토큰으로 분석하기 때문에 분석 대상 언어에 대한 지식이 필요 X<br>
<br>
## 과정
1. 어휘 집합 구축: 자주 등장하는 문자열 병합, 어휘 집합에 추가(원하는 어휘 집합 크기가 될 때 까지 반복)
2. 토큰화 : 토큰화 대상 문장의 각 어절에서 어휘 집합에 있는 서브워드가 포함되었을 때 항상 서브워드를 어절에서 분리


# BRE 어휘 집합 구축

### Pre-Tokenize
Corpus의 모든 문장을 공백으로 나누는 작업

1. Pre-Tokenize을 적용하고 단어의 빈도를 모두 세어서 초기의 어휘 집합 구성
b,g,h,n,p,s,u


### Pre-Tokenize 결과

| 토큰  | 빈도 |
|------|------|
| hug  | 10   |
| pug  | 5    |
| pun  | 12   |
| bun  | 4    |
| hugs | 5    |

### 초기 어휘 집합으로 다시 작성한 빈도표

| 토큰        | 빈도 |
|------------|------|
| h, u, g   | 10   |
| p, u, g   | 5    |
| p, u, n   | 12   |
| b, u, n   | 4    |
| h, u, g, s | 5    |

다시 두쌍으로 묶어서 bigram 생성 및 병합

| 바이그램 쌍 | 빈도 |
|------------|------|
| b, u      | 4    |
| g, s      | 5    |
| h, u      | 15   |
| p, u      | 17   |
| u, g      | 20   |
| u, n      | 16   |

u,g 가 20개로 제일 많아서 병합(u,g -> ug)

> b,g,h,n,p,s,u,ug

| 바이그램 쌍 | 빈도 |
|------------|------|
| b, u      | 4    |
| h, ug      | 15    |
| p, u      | 12   |
| p, ug      | 5   |
| u, n      | 16   |
| ug, s     | 5  |

u,n 가 16개로 제일 많아서 병합 (u,n -> un)

> b,g,h,n,p,s,u,ug,un

| 바이그램 쌍 | 빈도 |
|------------|------|
| b, un     | 4    |
| h, ug     | 15   |
| p, ug     | 5    |
| p, un     | 12   |
| ug, s     | 5    |

## BPE 어휘 집합 구축

> b, g, h, n, p, s, u, ug, un, hug

## BPE 어휘 집합은 고빈도 바이그램 쌍을 병합하는 방식으로 구축

처음 병합한 대상은 u, g, 두 번째는 u, n, 마지막은 h, ug였음을 확인할 수 있다.<br>
이 내용 그대로 merges.txt로 저장<br>
병합 우선순위(merges.txt)<br>
u g<br>
u n<br>
h ug<br>


# BPE 토큰화

어휘 집합(vocab.json), 병합 우선순위(merge.txt)가 있으면 토큰화 수행 가능
<br>
ex) pug bug mug 라는 문장을 토큰화

> pug bug mug -> pug,bug,mug

1. 문자 단위로 분리

> pug -> p,u,g

2. merge.txt 파일을 참고해 병합 우선순위를 부여

> p,u -> 우선 순위 없음
<br> u,g -> 1순위

3. u,g의 우선 순위가 높으므로 먼저 합친다.

> p,u,g -> p, ug

4. merge.txt 파일을 한번 더 참고해 병합 우선수위를 부여

> p,ug -> 우선 순위 없음

병합할 대상이 없어서 stop <br>
이 순서를 반복<br>
&lt;unk&gt; -> 미등록 토큰<br>
여기서 m이 어휘 집합에 없어서 미등록 토큰이 된다.

> pug bug mug -> p, ug, b, ug, &lt;unk&gt;, ug

# WordPiece

* Corpus에서 자주 등장한 문자열을 토큰으로 인식 -> BPE와 본질적으로 유사
* 어휘 집합을 구축할 때 문자열을 병합하는 기준이 다름
* 빈도를 기준으로 병합 x -> 병합했을 때 Corpus의 Likelihood를 가장 높이는 쌍을 병합

<br>
병합 후보가 a,b 일때, #a, #b, #ab는 각각 a,b,ab라는 문자열의 빈도수, n은 전체 글자 수를 가리킨다.
<br>
분자는 ab가 연이어 등장할 확률, 분모는 a,b가 각각 등장할 확률의 곱
<br>

$\frac{\left(\frac{\# ab}{n} \right)}{\left(\frac{\# a}{n} \right)} \times \left(\frac{\# b}{n} \right)$

이 수식의 값이 커지려면 a와 b가 서로 독립임을 가정했을 때보다 둘이 자주 동시에 등장해야 한다.<br>
워드피스에서는 병합 후보에 오른 쌍을 미리 병합해 보고 잃는 것과 가치 등을 판단한 후에 병합. (병합 대상 전체 후보들 가운데 위와 같이 계산한 값이 가장 높은 쌍을 합친다.)
<br>

워드피스는 어휘 집합(vocab.txt)만 가지고 토큰화
<br>
워드피스에서는 분석 대상 어절에 어휘 집합에 있는 서브워드가 포홤돼 있을 때 해당 서브워드를 어절에서 분리<br>
단, 이러한 서브워드 후보가 여럿 있을 경우 가장 긴 서브워드를 선택.
<br>
이후 어절의 나머지에서 어휘 집합에 있는 서브워드를 다시 찾고(최장 일치 기준), 또 분리
<br>
분석 대상 문자열에서 서브워드 후보가 하나도 없으면 해당 문자열 전체를 미등록 단어로 취급

In [2]:
from Korpora import Korpora

nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 43.3MB/s]                                                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 30.7MB/s]                                                             
